<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1-Introduction" data-toc-modified-id="1-Introduction-1">1 Introduction</a></span></li><li><span><a href="#2-Functions-for-classifying-images" data-toc-modified-id="2-Functions-for-classifying-images-2">2 Functions for classifying images</a></span></li><li><span><a href="#3-Data-Processing" data-toc-modified-id="3-Data-Processing-3">3 Data Processing</a></span><ul class="toc-item"><li><span><a href="#3.1-Image-classification-of-Funil-(time-series)" data-toc-modified-id="3.1-Image-classification-of-Funil-(time-series)-3.1">3.1 Image classification of Funil (time series)</a></span></li><li><span><a href="#3.2-Image-classification-of-Curuai-lake-(time-series)" data-toc-modified-id="3.2-Image-classification-of-Curuai-lake-(time-series)-3.2">3.2 Image classification of Curuai lake (time series)</a></span></li><li><span><a href="#3.3-Image-classification-of-ECP-(novelty-assessment)" data-toc-modified-id="3.3-Image-classification-of-ECP-(novelty-assessment)-3.3">3.3 Image classification of ECP (novelty assessment)</a></span><ul class="toc-item"><li><span><a href="#3.3.1-Without-using-probability" data-toc-modified-id="3.3.1-Without-using-probability-3.3.1">3.3.1 Without using probability</a></span></li><li><span><a href="#3.3.2-Using-probability-excluding-the-lowest-1%-values" data-toc-modified-id="3.3.2-Using-probability-excluding-the-lowest-1%-values-3.3.2">3.3.2 Using probability excluding the lowest 1% values</a></span></li><li><span><a href="#3.3.3-Retrieve-prob-values" data-toc-modified-id="3.3.3-Retrieve-prob-values-3.3.3">3.3.3 Retrieve prob values</a></span></li></ul></li><li><span><a href="#3.4-Image-classification-of-Funil-(novelty-assessment)" data-toc-modified-id="3.4-Image-classification-of-Funil-(novelty-assessment)-3.4">3.4 Image classification of Funil (novelty assessment)</a></span><ul class="toc-item"><li><span><a href="#3.4.1-Without-using-probability" data-toc-modified-id="3.4.1-Without-using-probability-3.4.1">3.4.1 Without using probability</a></span></li><li><span><a href="#3.4.2-Using-probability-excluding-the-lowest-1%-values" data-toc-modified-id="3.4.2-Using-probability-excluding-the-lowest-1%-values-3.4.2">3.4.2 Using probability excluding the lowest 1% values</a></span></li><li><span><a href="#3.4.3-Retrieve-prob-values" data-toc-modified-id="3.4.3-Retrieve-prob-values-3.4.3">3.4.3 Retrieve prob values</a></span></li></ul></li><li><span><a href="#3.5-Image-classification-of-Curuai-lake-(novelty-assessment)" data-toc-modified-id="3.5-Image-classification-of-Curuai-lake-(novelty-assessment)-3.5">3.5 Image classification of Curuai lake (novelty assessment)</a></span><ul class="toc-item"><li><span><a href="#3.5.1-Without-using-probability" data-toc-modified-id="3.5.1-Without-using-probability-3.5.1">3.5.1 Without using probability</a></span></li><li><span><a href="#3.5.2-Using-probability-excluding-the-lowest-1%-values" data-toc-modified-id="3.5.2-Using-probability-excluding-the-lowest-1%-values-3.5.2">3.5.2 Using probability excluding the lowest 1% values</a></span></li><li><span><a href="#3.5.3-Retrieve-prob-value" data-toc-modified-id="3.5.3-Retrieve-prob-value-3.5.3">3.5.3 Retrieve prob value</a></span></li></ul></li></ul></li></ul></div>

# 1 Introduction
<br>
This notebook classify the examples images to shown the algorithm classification of Sentinel-2 MSI

# 2 Functions for classifying images

In [1]:
# library used
from osgeo import gdal
from osgeo.gdalconst import *
import os
def image_to_arrays(image):
    '''
    This function read the output image from atmopheric correction and deglint, which
    have the following structure bands:
    0: B2
    1: B3
    2: B4
    3: B5
    4: B6
    Then, it returns the arrays of all bands.
    -----------------------
    image(str): image path
    -----------------------
    return: the five bands necessary for the OWT classification.

    '''
    
    # read image
    image =  gdal.Open(image, GA_ReadOnly)
    
    # read arrays
    bandas = image.ReadAsArray()

    # band selection
    b2 = bandas[0]
    b3 = bandas[1]
    b4 = bandas[2]
    b5 = bandas[3]
    b6 = bandas[4]


    return b2, b3, b4, b5, b6

In [2]:
# library used
import pandas as pd
import numpy as np
import pickle
import sklearn
def SVMC_image_classification(b2, b3, b4, b5, b6,
                              alg_path='00_Database/02_Algorithms/msi_svmc_owts_alldataset.obj',
                              novelty=False,
                              retrieve_prob=False,
                              msisvcprob={'OWT 1': 72,
                                          'OWT 2': 56,
                                          'OWT 3': 30,
                                          'OWT 4': 35,
                                          'OWT 5': 56,
                                          'OWT 6': 50,
                                          'OWT 7': 74,
                                          'OWT 8': 50}):
    '''
    This function utilize the Support Vector Machine Classification (SVMC) built on
    Scikit-learn for classifying arrays extracted from Sentinel-2 images.
    -------------------------
    b2 (np.array): Remote sensing reflectance (Rrs) of B2 band.
    b3 (np.array): Rrs of B3 band.
    b4 (np.array): Rrs of B4 band.
    b5 (np.array): Rrs of B5 band.
    b6 (np.array): Rrs of B6 band.
    alg_path (str): Algorithm path for the list with shape and intensity classification.
    novelty (boolean): Use probability estimations for detectin novel OWT based on specific thresholds. 
    retrieve_prob (boolean): Retrieve the map of progability values instead of OWTs.
    --------------------------
    return a classified array
    
    '''

    
    # Open the SVMC algorithm
    msi_svmc_alg = open(alg_path, 'rb') 
    msi_svmc_alg = pickle.load(msi_svmc_alg)
    model = msi_svmc_alg[0]
    modelint = msi_svmc_alg[1]
    
    # normalize bands by its sum
    bandasoma = b2+b3+b4+b5+b6
    bandasoma[bandasoma==0] = -32 # 0 values crashes the division
    b2norm = b2 / bandasoma
    b2 = None # dump cache
    b3norm = b3 / bandasoma
    b4norm = b4 / bandasoma
    b4 = None # dump cache
    b5norm = b5 / bandasoma
    b5 = None # dump cache
    b6norm = b6 / bandasoma
    b6 = None # dump cache
    bandasoma = None # dump cache
    
    # negative values are removed because they may crash the classifications
    b2norm[b2norm<0] = 0
    b3norm[b3norm<0] = 0
    b4norm[b4norm<0] = 0
    b5norm[b5norm<0] = 0
    b6norm[b6norm<0] = 0
    b3[b3<0] = 0

    # reshape array to 1d (needed for the scikit-learn algorithm)
    d2shape = (b5norm.shape[0], b5norm.shape[1])
    d1shape = b5norm.size
    b2norm1d = b2norm.reshape(d1shape)
    b3norm1d = b3norm.reshape(d1shape)
    b4norm1d = b4norm.reshape(d1shape)
    b5norm1d = b5norm.reshape(d1shape)
    b6norm1d = b6norm.reshape(d1shape)
    b31d = b3.reshape(d1shape)
    threeband1d = np.vstack([b2norm1d, b3norm1d, b4norm1d, b5norm1d, b6norm1d, b31d]).transpose()
    threeband1d = pd.DataFrame(threeband1d)
    
    b2norm1d = None
    b3norm1d = None
    b4norm1d = None
    b5norm1d = None
    b6norm1d = None
    b31d = None
    b2norm = None # dump chache
    b3norm = None # dump chache
    b4norm = None # dump chache
    b5norm = None # dump chache
    b6norm = None # dump chache
    b3 = None # dump chache
    
    # drop values below 0 (mask) to speed up classification
    target = threeband1d[threeband1d>0].dropna()
    
    # classification
    # 1 shape classification
    predicted = model.predict(target.iloc[:,0:-1])
    predicted = pd.DataFrame(predicted,
                             index=target.index,
                             columns=['predicted'])
    if novelty == True:
        prob = pd.DataFrame(model.predict_proba(target.iloc[:,0:-1]),
                            index=target.index,
                            columns=model.classes_)
        predicted = predicted.join(prob)
        prob = None
    
    # 2 intensity classification
    owt789id = predicted[predicted['predicted'] == 'OWT 678'].index
    predicteddt = modelint.predict(target.iloc[:,-1:].loc[owt789id,:].values)
    predicteddt = pd.DataFrame(predicteddt,
                               index=owt789id,
                               columns=['predicted'])
    if novelty == True:
        predicteddtprob = pd.DataFrame(modelint.predict_proba(target.iloc[:,-1:].loc[owt789id,:].values),
                                       index=owt789id,
                                       columns=modelint.classes_)
        predicteddtprob = predicteddtprob.multiply(predicted['OWT 678'], axis=0).dropna(axis=0, how='all')
        predicteddt = predicteddt.join(predicteddtprob)
        predicteddtprob = None
    
    # Concat shape and intensity classification
    predicted.update(predicteddt)
    if novelty == True:
        predicted = predicted.join(predicteddt.iloc[:,1:])
        predicted = predicted.drop('OWT 678', axis=1)
        
    predicteddt = None
    predicteddtprob = None
    
    # classify as novel the predictions with lower probability than the threshold set
    if novelty == True and retrieve_prob==False:
        
        for x in predicted.columns[1:]:

            try:
                id_to_drop = predicted[(predicted[x] < msisvcprob[x]/100) & (predicted['predicted'] == x)].index
                predicted.loc[id_to_drop, 'predicted'] = np.nan
            
            except:
                print('Novelty detection failed to: ', x)
                print('Probably, this OWT does not occur in the image')

    elif novelty == True and retrieve_prob==True:
        
        predicted = predicted.max(axis=1)*100
        predicted.name = 'predicted'
    if retrieve_prob==False:
        
        # replace OWTs names by numbers, for saving in a tiff file
        predicted = predicted.replace(['OWT 1', 'OWT 2', 'OWT 3', 'OWT 4','OWT 5', 'OWT 6', 'OWT 7','OWT 8', 'OWT 9'],
                                      [1, 2, 3, 4, 5, 6, 7, 8, 9])
    
    # generate the 2D array classification
    threeband1d = threeband1d.join(predicted)
    classified = threeband1d['predicted'].values.reshape(d2shape)
    
    return classified


In [3]:
# library used
from osgeo import gdal

def array_to_image(classified, i, path, geotransform, projection):
    '''
    This function convert a array to tiff
    -----------------------------
    classified (np.array): an image array.
    i (str): image name.
    path: path for saving the tiff image.
    geotransform: Geotransform for saving the image (e.g.: image.GetGeoTransform()).
    projection: Projection for saving the image (e.g.: image.GetProjection())
    '''   
    # save results
    # set image dimensions
    image_size = classified.shape
    nx = image_size[0]
    ny = image_size[1]

    # create the classified raster file
    dst_ds = gdal.GetDriverByName('GTiff').Create(path+'/'+i+'_owt.tif', ny, nx, 1, gdal.GDT_Int16)

    dst_ds.SetGeoTransform(geotransform)    # specify coords
    dst_ds.SetProjection(projection) # export coords to file       
    dst_ds.GetRasterBand(1).WriteArray(classified)   # write r-band to the raster
    dst_ds.GetRasterBand(1).SetNoDataValue(0) 
    dst_ds.FlushCache()                     # write to disk


# 3 Data Processing

## 3.1 Image classification of Funil (time series)

In [4]:
#library
import glob

# add path with bands and folder to save images
path = '00_Database/03_Images/03_Funil/l2/'
owt_path = '00_Database/03_Images/03_Funil/l3_owts'

for img in glob.glob(path+'*.tif'):
    
    b2, b3, b4, b5, b6 = image_to_arrays(img)
    b2 = b2*0.0001
    b3 = b3*0.0001
    b4 = b4*0.0001
    b5 = b5*0.0001
    b6 = b6*0.0001
    img_shape = b3.shape

    # split arrays for reduce memory usage in the classification
    b2 = np.split(b2,10)
    b3 = np.split(b3,10)
    b4 = np.split(b4,10)
    b5 = np.split(b5,10)
    b6 = np.split(b6,10)


    # image classification
    owt_image = {}
    for x in range(0,10):
        owt_chunck = SVMC_image_classification(b2[x],
                                              b3[x],
                                              b4[x],
                                              b5[x],
                                              b6[x], novelty=True)

        owt_image[x] = owt_chunck
        owt_chunck = None

    owt_image = np.concatenate((owt_image[0],
                                owt_image[1],
                                owt_image[2],
                                owt_image[3],
                                owt_image[4],
                                owt_image[5],
                                owt_image[6],
                                owt_image[7],
                                owt_image[8],
                                owt_image[9],), axis=0)


    img_ref = gdal.Open(img, GA_ReadOnly)

    geotransform = img_ref.GetGeoTransform()
    proj = img_ref.GetProjection()


    array_to_image(owt_image,
                   img.split('/')[-1].split('.')[0].split('B2')[0],
                   owt_path,
                   geotransform,
                   proj)

## 3.2 Image classification of Curuai lake (time series)

In [5]:
# add path with bands and folder to save images
path = '00_Database/03_Images/04_Curuai/l2/'
owt_path = '00_Database/03_Images/04_Curuai/l3_owts'

for img in glob.glob(path+'*.tif'):
    
    b2, b3, b4, b5, b6 = image_to_arrays(img)
    b2 = b2*0.0001
    b3 = b3*0.0001
    b4 = b4*0.0001
    b5 = b5*0.0001
    b6 = b6*0.0001
    img_shape = b3.shape

    # split arrays for reduce memory usage in the classification
    b2 = np.split(b2,10)
    b3 = np.split(b3,10)
    b4 = np.split(b4,10)
    b5 = np.split(b5,10)
    b6 = np.split(b6,10)


    # image classification
    owt_image = {}
    for x in range(0,10):
        owt_chunck = SVMC_image_classification(b2[x],
                                              b3[x],
                                              b4[x],
                                              b5[x],
                                              b6[x], novelty=True)

        owt_image[x] = owt_chunck
        owt_chunck = None

    owt_image = np.concatenate((owt_image[0],
                                owt_image[1],
                                owt_image[2],
                                owt_image[3],
                                owt_image[4],
                                owt_image[5],
                                owt_image[6],
                                owt_image[7],
                                owt_image[8],
                                owt_image[9],), axis=0)


    img_ref = gdal.Open(img, GA_ReadOnly)

    geotransform = img_ref.GetGeoTransform()
    proj = img_ref.GetProjection()


    array_to_image(owt_image,
                   img.split('/')[-1].split('.')[0].split('B2')[0],
                   owt_path,
                   geotransform,
                   proj)

## 3.3 Image classification of ECP (novelty assessment)

### 3.3.1 Without using probability

In [6]:
# add path and image name
path = '00_Database/03_Images/02_ECP'
img = 'T22JGS_20190724T132241B2B3B4B5b6.tif'
b2, b3, b4, b5, b6 = image_to_arrays(path+'/'+img)
b2 = b2*0.0001
b3 = b3*0.0001
b4 = b4*0.0001
b5 = b5*0.0001
b6 = b6*0.0001
img_shape = b3.shape

# split arrays for reduce memory usage in the classification
b2 = np.split(b2,10)
b3 = np.split(b3,10)
b4 = np.split(b4,10)
b5 = np.split(b5,10)
b6 = np.split(b6,10)


# image classification
owt_image = {}
for x in range(0,10):
    owt_chunck = SVMC_image_classification(b2[x],
                                          b3[x],
                                          b4[x],
                                          b5[x],
                                          b6[x], novelty=False)

    owt_image[x] = owt_chunck
    owt_chunck = None

owt_image = np.concatenate((owt_image[0],
                            owt_image[1],
                            owt_image[2],
                            owt_image[3],
                            owt_image[4],
                            owt_image[5],
                            owt_image[6],
                            owt_image[7],
                            owt_image[8],
                            owt_image[9],), axis=0)


img_ref = gdal.Open(path+'/'+img, GA_ReadOnly)

geotransform = img_ref.GetGeoTransform()
proj = img_ref.GetProjection()


array_to_image(owt_image,
               img.split('/')[-1].split('.')[0].split('B2')[0]+'_withoutnovelty',
               path,
               geotransform,
               proj)

### 3.3.2 Using probability excluding the lowest 1% values

In [7]:
# add path and image name
path = '00_Database/03_Images/02_ECP'
img = 'T22JGS_20190724T132241B2B3B4B5b6.tif'
b2, b3, b4, b5, b6 = image_to_arrays(path+'/'+img)
b2 = b2*0.0001
b3 = b3*0.0001
b4 = b4*0.0001
b5 = b5*0.0001
b6 = b6*0.0001
img_shape = b3.shape

# split arrays for reduce memory usage in the classification
b2 = np.split(b2,10)
b3 = np.split(b3,10)
b4 = np.split(b4,10)
b5 = np.split(b5,10)
b6 = np.split(b6,10)


# image classification
owt_image = {}
for x in range(0,10):
    owt_chunck = SVMC_image_classification(b2[x],
                                          b3[x],
                                          b4[x],
                                          b5[x],
                                          b6[x], novelty=True)

    owt_image[x] = owt_chunck
    owt_chunck = None

owt_image = np.concatenate((owt_image[0],
                            owt_image[1],
                            owt_image[2],
                            owt_image[3],
                            owt_image[4],
                            owt_image[5],
                            owt_image[6],
                            owt_image[7],
                            owt_image[8],
                            owt_image[9],), axis=0)


img_ref = gdal.Open(path+'/'+img, GA_ReadOnly)

geotransform = img_ref.GetGeoTransform()
proj = img_ref.GetProjection()


array_to_image(owt_image,
               img.split('/')[-1].split('.')[0].split('B2')[0],
               path,
               geotransform,
               proj)

### 3.3.3 Retrieve prob values

In [8]:
# add path and image name
path = '00_Database/03_Images/02_ECP'
img = 'T22JGS_20190724T132241B2B3B4B5b6.tif'
b2, b3, b4, b5, b6 = image_to_arrays(path+'/'+img)
b2 = b2*0.0001
b3 = b3*0.0001
b4 = b4*0.0001
b5 = b5*0.0001
b6 = b6*0.0001
img_shape = b3.shape

# split arrays for reduce memory usage in the classification
b2 = np.split(b2,10)
b3 = np.split(b3,10)
b4 = np.split(b4,10)
b5 = np.split(b5,10)
b6 = np.split(b6,10)


# image classification
owt_image = {}
for x in range(0,10):
    owt_chunck = SVMC_image_classification(b2[x],
                                          b3[x],
                                          b4[x],
                                          b5[x],
                                          b6[x],
                                           novelty=True,
                                          retrieve_prob=True)

    owt_image[x] = owt_chunck
    owt_chunck = None

owt_image = np.concatenate((owt_image[0],
                            owt_image[1],
                            owt_image[2],
                            owt_image[3],
                            owt_image[4],
                            owt_image[5],
                            owt_image[6],
                            owt_image[7],
                            owt_image[8],
                            owt_image[9],), axis=0)


img_ref = gdal.Open(path+'/'+img, GA_ReadOnly)

geotransform = img_ref.GetGeoTransform()
proj = img_ref.GetProjection()


array_to_image(owt_image,
               img.split('/')[-1].split('.')[0].split('B2')[0]+'_owtprobvalues',
               path,
               geotransform,
               proj)

## 3.4 Image classification of Funil (novelty assessment)

### 3.4.1 Without using probability

In [9]:
# add path and image name
path = '00_Database/03_Images/03_Funil/l2'
img = 'T23KNR_20180820T131239B2B3B4B5b6.tif'
owt_path = '00_Database/03_Images/03_Funil/l3_owts_without_novelty'

b2, b3, b4, b5, b6 = image_to_arrays(path+'/'+img)
b2 = b2*0.0001
b3 = b3*0.0001
b4 = b4*0.0001
b5 = b5*0.0001
b6 = b6*0.0001
img_shape = b3.shape

# split arrays for reduce memory usage in the classification
b2 = np.split(b2,10)
b3 = np.split(b3,10)
b4 = np.split(b4,10)
b5 = np.split(b5,10)
b6 = np.split(b6,10)


# image classification
owt_image = {}
for x in range(0,10):
    owt_chunck = SVMC_image_classification(b2[x],
                                          b3[x],
                                          b4[x],
                                          b5[x],
                                          b6[x], novelty=False)

    owt_image[x] = owt_chunck
    owt_chunck = None

owt_image = np.concatenate((owt_image[0],
                            owt_image[1],
                            owt_image[2],
                            owt_image[3],
                            owt_image[4],
                            owt_image[5],
                            owt_image[6],
                            owt_image[7],
                            owt_image[8],
                            owt_image[9],), axis=0)


img_ref = gdal.Open(path+'/'+img, GA_ReadOnly)

geotransform = img_ref.GetGeoTransform()
proj = img_ref.GetProjection()


array_to_image(owt_image,
               img.split('/')[-1].split('.')[0].split('B2')[0]+'_withoutnovelty',
               owt_path,
               geotransform,
               proj)

### 3.4.2 Using probability excluding the lowest 1% values
<br>
This was already done in the 3.1 section. Thus, the same image is used.

### 3.4.3 Retrieve prob values

In [10]:
# add path and image name
path = '00_Database/03_Images/03_Funil/l2'
img = 'T23KNR_20180820T131239B2B3B4B5b6.tif'
owt_path = '00_Database/03_Images/03_Funil/l3_owts_without_novelty'

b2, b3, b4, b5, b6 = image_to_arrays(path+'/'+img)
b2 = b2*0.0001
b3 = b3*0.0001
b4 = b4*0.0001
b5 = b5*0.0001
b6 = b6*0.0001
img_shape = b3.shape

# split arrays for reduce memory usage in the classification
b2 = np.split(b2,10)
b3 = np.split(b3,10)
b4 = np.split(b4,10)
b5 = np.split(b5,10)
b6 = np.split(b6,10)


# image classification
owt_image = {}
for x in range(0,10):
    owt_chunck = SVMC_image_classification(b2[x],
                                          b3[x],
                                          b4[x],
                                          b5[x],
                                          b6[x],
                                           novelty=True,
                                          retrieve_prob=True)

    owt_image[x] = owt_chunck
    owt_chunck = None

owt_image = np.concatenate((owt_image[0],
                            owt_image[1],
                            owt_image[2],
                            owt_image[3],
                            owt_image[4],
                            owt_image[5],
                            owt_image[6],
                            owt_image[7],
                            owt_image[8],
                            owt_image[9],), axis=0)


img_ref = gdal.Open(path+'/'+img, GA_ReadOnly)

geotransform = img_ref.GetGeoTransform()
proj = img_ref.GetProjection()


array_to_image(owt_image,
               img.split('/')[-1].split('.')[0].split('B2')[0]+'owt_prob_values.tif',
               owt_path,
               geotransform,
               proj)

## 3.5 Image classification of Curuai lake (novelty assessment)

### 3.5.1 Without using probability

In [11]:
# add path and image name
path = '00_Database/03_Images/04_Curuai/l2'
img = 'T21MXT_20190818T141051B2B3B4B5b6.tif'
owt_path = '00_Database/03_Images/04_Curuai/l3_owts_without_novelty'

b2, b3, b4, b5, b6 = image_to_arrays(path+'/'+img)
b2 = b2*0.0001
b3 = b3*0.0001
b4 = b4*0.0001
b5 = b5*0.0001
b6 = b6*0.0001
img_shape = b3.shape

# split arrays for reduce memory usage in the classification
b2 = np.split(b2,10)
b3 = np.split(b3,10)
b4 = np.split(b4,10)
b5 = np.split(b5,10)
b6 = np.split(b6,10)


# image classification
owt_image = {}
for x in range(0,10):
    owt_chunck = SVMC_image_classification(b2[x],
                                          b3[x],
                                          b4[x],
                                          b5[x],
                                          b6[x], novelty=False)

    owt_image[x] = owt_chunck
    owt_chunck = None

owt_image = np.concatenate((owt_image[0],
                            owt_image[1],
                            owt_image[2],
                            owt_image[3],
                            owt_image[4],
                            owt_image[5],
                            owt_image[6],
                            owt_image[7],
                            owt_image[8],
                            owt_image[9],), axis=0)


img_ref = gdal.Open(path+'/'+img, GA_ReadOnly)

geotransform = img_ref.GetGeoTransform()
proj = img_ref.GetProjection()


array_to_image(owt_image,
               img.split('/')[-1].split('.')[0].split('B2')[0]+'owt.tif',
               owt_path,
               geotransform,
               proj)

### 3.5.2 Using probability excluding the lowest 1% values
<br>
This was already done in the 3.2 section. Thus, the same image is used.

### 3.5.3 Retrieve prob value

In [12]:
# add path and image name
path = '00_Database/03_Images/04_Curuai/l2'
img = 'T21MXT_20190818T141051B2B3B4B5b6.tif'
owt_path = '00_Database/03_Images/04_Curuai/l3_owts_without_novelty'

b2, b3, b4, b5, b6 = image_to_arrays(path+'/'+img)
b2 = b2*0.0001
b3 = b3*0.0001
b4 = b4*0.0001
b5 = b5*0.0001
b6 = b6*0.0001
img_shape = b3.shape

# split arrays for reduce memory usage in the classification
b2 = np.split(b2,10)
b3 = np.split(b3,10)
b4 = np.split(b4,10)
b5 = np.split(b5,10)
b6 = np.split(b6,10)


# image classification
owt_image = {}
for x in range(0,10):
    owt_chunck = SVMC_image_classification(b2[x],
                                          b3[x],
                                          b4[x],
                                          b5[x],
                                          b6[x],
                                           novelty=True,
                                          retrieve_prob=True)

    owt_image[x] = owt_chunck
    owt_chunck = None

owt_image = np.concatenate((owt_image[0],
                            owt_image[1],
                            owt_image[2],
                            owt_image[3],
                            owt_image[4],
                            owt_image[5],
                            owt_image[6],
                            owt_image[7],
                            owt_image[8],
                            owt_image[9],), axis=0)


img_ref = gdal.Open(path+'/'+img, GA_ReadOnly)

geotransform = img_ref.GetGeoTransform()
proj = img_ref.GetProjection()


array_to_image(owt_image,
               img.split('/')[-1].split('.')[0].split('B2')[0]+'owt_prob_values.tif',
               owt_path,
               geotransform,
               proj)